# Extract

In [2]:
# Importing libraries
import csv
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [3]:
df_artist=pd.read_csv('./csv/artists_mod.csv')
df_tracks=pd.read_csv('./csv/tracks_mod.csv')
print(df_artist.shape)

(1162095, 5)


# Transform

## Tracks

In [4]:
df_tracks = df_tracks[df_tracks['popularity'].notna()] ## KEEP TRACKS WITH POPULARITY
df_tracks.dropna(inplace=True) # quitar los NaN
df_tracks['duration_ms']=df_tracks['duration_ms']/1000 # Tener duración en segundos

In [5]:
df_tracks.rename(columns={"duration_ms": "duration_s"}, inplace=True) # cambiar nombre columna

In [6]:
df_tracks.dropna(inplace=True) # quita las canciones con vacios para asegurar calidad de datos
df_tracks.drop_duplicates(inplace=True) # borrar canciones duplicadas

In [7]:
variables=['id','name','popularity','duration_s','explicit','id_artists','danceability','energy','loudness','speechiness','acousticness','tempo'] # se eligen las variables deseadas
df_tracks=df_tracks[variables] # selección de variables a utilizar

In [8]:
from sklearn import preprocessing
import ast

df_tracks['loudness']=abs(df_tracks['loudness']) # se obtiene el valor absoluto del loudness
df_tracks_ids= df_tracks[['id','name','id_artists']] # se obtiene un nuevo dataframe con los datos deseados

min_max_scaler = preprocessing.MinMaxScaler() # se normaliza para subir a postgresql
df_tracks_norm = min_max_scaler.fit_transform(df_tracks[['popularity','duration_s','explicit','danceability','energy','loudness','speechiness','acousticness','tempo']])
df_tracks_norm = pd.DataFrame(df_tracks_norm, columns=['popularity','duration_s','explicit','danceability','energy','loudness','speechiness','acousticness','tempo'])
df_tracks_final= pd.merge(df_tracks_ids, df_tracks_norm, left_index=True, right_index=True) ## este es el que se exporta


df_tracks_final['id_artists']=df_tracks_final["id_artists"].apply(ast.literal_eval)
df_tracks_final=df_tracks_final.explode('id_artists') # una canción por artista


df_tracks_final.rename(columns={"name": "name_tracks"}, inplace = True)
df_tracks_final['id_artists'] = df_tracks_final['id_artists'].replace({'\'': ''},regex=True).replace({"'": ''},regex=True)
df_tracks_final['name_tracks'] = df_tracks_final['name_tracks'].replace({'\'': ''},regex=True).replace({"'": ''},regex=True)

df_tracks_final

,id,name_tracks,id_artists,popularity,duration_s,explicit,danceability,energy,loudness,speechiness,acousticness,tempo
0,35iwgR4jXetI318WEWsa1Q,Carve,45tIt06XoI0Iio4LBEVpls,0.06,0.021994,0.5,0.650858,0.4450,0.222235,0.464470,0.676707,0.430587
1,021ht4sdgPcrDgSk7JTbKY,Capítulo 2.16 - Banquero Anarquista,14jtPCOoNZwquk5wd9DxrY,0.00,0.016885,0.5,0.701312,0.2630,0.368881,0.985582,0.800201,0.418916
2,07A5yehtSnoedViJAZkNnc,Vivo para Quererte - Remasterizado,5LiOoJbxVSAMkBS2fUm3X2,0.00,0.031737,0.5,0.437941,0.1770,0.352946,0.052729,0.997992,0.535582
4,08y9GfoqCWfOGsKdwojr5e,Lady of the Evening,3BiJGZsyX9sJchTqcSA7Su,0.00,0.031255,0.5,0.229062,0.2610,0.205650,0.039341,0.997992,0.488245
5,0BRXJHRNGQ3W4v9frnSfhu,Ave Maria,3BiJGZsyX9sJchTqcSA7Su,0.00,0.023340,0.5,0.514632,0.3550,0.213818,0.127703,0.968876,0.352162
...,...,...,...,...,...,...,...,...,...,...,...,...
348056,4Vv6QZb1Vyneb6osDKL2k6,Santa Lucia,1dpnxi6xgoB2kaRYnnoatZ,0.70,0.034792,0.5,0.497477,0.4610,0.141995,0.046962,0.848394,0.211140
348059,2lZkDozWgOQhamMelasD3t,Diga Que Le Debo,3baxdZ69449O6BMd7yuYXv,0.67,0.014090,0.5,0.614531,0.0172,0.476173,0.121524,1.000000,0.281795
348061,7hLbRnIQNqZQ6e46NrrBF8,El Vampiro,3rNIGWm5Dz3ni9lRcDgJip,0.47,0.027243,0.5,0.683148,0.9700,0.056388,0.045932,0.134538,0.575039
348063,6lD0i7g0vJlRzcCPrsG3IN,Fuego en la Piel,7gyFPwPjI8N3wIyTghT7xw,0.50,0.045377,0.5,0.565086,0.5180,0.124444,0.030072,0.788153,0.541652


## Géneros

In [9]:
import ast

# Aplicar ast.literal_eval() a cada fila de la columna "mi_columna"
df_artist["genres"] = df_artist["genres"].apply(ast.literal_eval)
df3 = pd.DataFrame(df_artist['genres'].to_list())
df3 = df3[0]
df_genres = pd.concat([df_artist, df3], axis=1)
df_genres= df_genres[[0]]
df_genres= df_genres.drop_duplicates(subset=[0], keep = 'first')
df_genres.rename(columns={0: "genero"}, inplace = True)
df_genres.dropna(inplace = True)
df_genres['genre_id'] =range(1, len(df_genres)+1) #df_genres[['genero']].sum(axis=1).map(hash).abs()
df_genres=df_genres[['genre_id','genero']]
df_genres['genero'] = df_genres['genero'].replace({'\'': ''},regex=True)

In [10]:
#df_genres['genre_id'].astype(int)
df_genres

,genre_id,genero
45,1,carnaval cadiz
136,2,classical harp
141,3,classical contralto
142,4,british choir
144,5,childrens story
...,...,...
816992,5359,thrash metal
840273,5360,ugandan traditional
1045510,5361,viking metal
1118346,5362,metal piauiense


## Artistas

In [11]:
# Aplicar ast.literal_eval() a cada fila de la columna "mi_columna"
df_artists = pd.concat([df_artist, df3], axis=1)
df_artists.rename(columns={0: "genero"}, inplace = True)
df_artists = df_artists[df_artists['genero'].notna()]
df_artists.drop(['genres'], axis = 1, inplace= True) # remove column (genres)
df_artists

,id,followers,name,popularity,genero
45,0VLMVnVbJyJ4oyZs2L3Yl2,71.0,Las Viudas De Los Bisabuelos,6,carnaval cadiz
46,0dt23bs4w8zx154C5xdVyl,63.0,Los De Capuchinos,5,carnaval cadiz
47,0pGhoB99qpEJEsBQxgaskQ,64.0,Los “Pofesionales”,7,carnaval cadiz
48,3HDrX2OtSuXLW5dLR85uN3,53.0,Los Que No Paran De Rajar,6,carnaval cadiz
136,22mLrN5fkppmuUPsHx6i2G,59.0,Vera Dulova,3,classical harp
...,...,...,...,...,...
1162087,0vZbvoyHDSwTGY4v1oUo31,1682.0,Josh Sneed,20,clean comedy
1162088,3V4ODgUObsEHFX6GQUy8xl,419.0,Matt McCarthy,9,deep comedy
1162090,3cOzi726Iav1toV2LRVEjp,4831.0,Ali Siddiq,34,black comedy
1162093,5nvjpU3Y7L6Hpe54QuvDjy,2357.0,Donnell Rawlings,15,black comedy


In [12]:
df_artists=pd.merge(df_artists,df_genres, on='genero', how='left')
df_artists = df_artists[df_artists['genre_id'].notna()]
df_artists

,id,followers,name,popularity,genero,genre_id
0,0VLMVnVbJyJ4oyZs2L3Yl2,71.0,Las Viudas De Los Bisabuelos,6,carnaval cadiz,1.0
1,0dt23bs4w8zx154C5xdVyl,63.0,Los De Capuchinos,5,carnaval cadiz,1.0
2,0pGhoB99qpEJEsBQxgaskQ,64.0,Los “Pofesionales”,7,carnaval cadiz,1.0
3,3HDrX2OtSuXLW5dLR85uN3,53.0,Los Que No Paran De Rajar,6,carnaval cadiz,1.0
4,22mLrN5fkppmuUPsHx6i2G,59.0,Vera Dulova,3,classical harp,2.0
...,...,...,...,...,...,...
305590,0vZbvoyHDSwTGY4v1oUo31,1682.0,Josh Sneed,20,clean comedy,231.0
305591,3V4ODgUObsEHFX6GQUy8xl,419.0,Matt McCarthy,9,deep comedy,2412.0
305592,3cOzi726Iav1toV2LRVEjp,4831.0,Ali Siddiq,34,black comedy,3448.0
305593,5nvjpU3Y7L6Hpe54QuvDjy,2357.0,Donnell Rawlings,15,black comedy,3448.0


In [13]:
df_artists = df_artists[df_artists['genre_id'].notna()]
df_artists = df_artists[df_artists['followers'].notna()]
df_artists = df_artists[df_artists['popularity'].notna()]
df_artists[df_artists['genre_id'].isnull()]
df_artists.columns = ['id', 'followers', 'name_art','popularity','genero', 'genre_id']
df_artists['name_art'] = df_artists['name_art'].replace({'\'': ''},regex=True)
df_artists = df_artists.reindex(columns=['id', 'followers', 'genre_id', 'name_art', 'popularity'])
df_artists

,id,followers,genre_id,name_art,popularity
0,0VLMVnVbJyJ4oyZs2L3Yl2,71.0,1.0,Las Viudas De Los Bisabuelos,6
1,0dt23bs4w8zx154C5xdVyl,63.0,1.0,Los De Capuchinos,5
2,0pGhoB99qpEJEsBQxgaskQ,64.0,1.0,Los “Pofesionales”,7
3,3HDrX2OtSuXLW5dLR85uN3,53.0,1.0,Los Que No Paran De Rajar,6
4,22mLrN5fkppmuUPsHx6i2G,59.0,2.0,Vera Dulova,3
...,...,...,...,...,...
305590,0vZbvoyHDSwTGY4v1oUo31,1682.0,231.0,Josh Sneed,20
305591,3V4ODgUObsEHFX6GQUy8xl,419.0,2412.0,Matt McCarthy,9
305592,3cOzi726Iav1toV2LRVEjp,4831.0,3448.0,Ali Siddiq,34
305593,5nvjpU3Y7L6Hpe54QuvDjy,2357.0,3448.0,Donnell Rawlings,15


# Load

## Postgresql

In [17]:
import psycopg2

try:
    connection=psycopg2.connect(
        host='localhost',
        user='myuser',
        password='mypassword',
        database='mydatabase'
    )
    #print("Conexión Iniciada")
    cursor=connection.cursor()
    cursor.execute("SELECT version()")
    row=cursor.fetchone()
    #print(row)
    
  
    # Insert Dataframe into SQL Server:
    for index, row in df_genres.iterrows():
        consulta=f""" INSERT INTO Gender (id_gender,name_gender) VALUES({row.genre_id},'{row.genero}') """
        cursor.execute(consulta)
        connection.commit()
        
    for index, row in df_artists.iterrows():
        consulta=f""" INSERT INTO Artists (id_artists,followers,id_gender,name_artists,popularity_artists) VALUES('{row.id}',{row.followers},{row.genre_id},'{row.name_art}',{row.popularity}) """
        cursor.execute(consulta)
        connection.commit()
        
    
    for index, row in df_tracks_final.iterrows(): 
        consulta=f""" INSERT INTO Tracks (ID_tracks,name_tracks,popularity_tracks,duration_ms,explicit,id_artists,danceability,energy,loudness,speechiness,acousticness,tempo) VALUES('{row.id}', '{row.name_tracks}',{row.popularity},{row.duration_s},{row.explicit},'{row.id_artists}',{row.danceability},{row.energy},{row.loudness},{row.speechiness},{row.acousticness},{row.tempo}) """
        #print(consulta)
        cursor.execute(consulta)
        connection.commit()
        
        
    # Ejecución de una consulta
    cursor.execute("SELECT * FROM Gender")
    # Recuperación de los resultados
    rows = cursor.fetchall()
    # Impresión de los resultados
    #for row in rows:
     #   print(row)

except Exception as ex:
    print(ex)
    
finally:
    connection.close()
    print("Conexión Finalizada")
    

Conexión Finalizada
